In [1]:
import pandas as pd
import numpy as np

In [2]:
from pymongo import MongoClient
import pprint

In [3]:
# Connect to the hosted MongoDB instance
client = MongoClient()

In [4]:
db = client.Street_Advisor

In [5]:
reviews = db.neighborhood

In [6]:
allreviews = reviews.find({},{'_id': 0})
reviewlist=[]
for review in allreviews:
    reviewlist.append(review)

In [7]:
df=pd.DataFrame(reviewlist)

In [9]:
df = df[['reviews','name']]
df[0:10]

,reviews,name
0,[\nOk I’m gonna let you in on one of my runnin...,Alki
1,[\nWant a neighborhood close to the city with ...,Arbor Heights
2,"[\nLocated just north of Interstate 90, betwee...",Atlantic
3,[\nOk I had to write a review for Ballard just...,Ballard
4,"[\nThe area of Beacon Hill, known more specifi...",Beacon Hill
5,[\nSome of my colleagues and I occasionally li...,Belltown
6,[\nWe looked at many Shoreline and North Seatt...,Briarcrest
7,[\nCarkeek park in Broadview is one of our fav...,Broadview Bitter Lake
8,[\nThe Epicenter of Seattle Culture\n\nA place...,Broadway (Capitol Hill)
9,"[\nLake City has some junky parts, where you d...",Cedar Park / Meadowbrook


In [ ]:
#Remove the neighborhoods dont have reviews

In [10]:
empty_review_list=[]
for i in range(df.shape[0]):
    if len(df['reviews'][i])==0:
        empty_review_list.append(df['name'][i])
empty_review_list

[u'Parkwood', u'Ridgecrest', u'Westminster Triangle']

In [ ]:
# reset index

In [11]:
for i in empty_review_list:
    df = df[df.name != i]

In [17]:
df.reset_index(drop=True,inplace=True)

In [19]:
df.drop('index',axis=1,inplace=True)

In [21]:
df[40:50]

,reviews,name
40,[\nThe Mt. Baker community is as different eco...,Mount Baker
41,[\nJust moved here recently because of the qua...,North Admiral
42,"[\nThere’s nowhere in Seattle that is as nice,...",North Beach Blue Ridge
43,[\nNorth Beacon Hill is another section of the...,North Beacon Hill
44,"[\nEveryone wants to live in the forest, well ...",North Delridge
45,[\nNorth Queen Anne has some lovely houseboats...,North Queen Anne
46,"[\nNorthgate is one of the biggest, most north...",Northgate
47,[\nThe greatest thing about the Olympia Hills ...,Olympic Hills / Victory Heights
48,[\nWe went to Phinney Ridge neighborhood to go...,Phinney Ridge / Greenwood
49,"[\nAfter the Space Needle, Pike Place Market i...",Pike Market


In [ ]:
# TFID Vectorizer

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [23]:
import string
punctuations_=set(string.punctuation)
textreview=[]
for i in range(df.shape[0]):
    textreview.append(''.join(word for word in df['reviews'][i] if word not in punctuations_))

In [52]:
len(textreview)

71

In [80]:
neighborhoods_list = df['name']
set(neighborhoods_list)

{u'Alki',
 u'Arbor Heights',
 u'Atlantic',
 u'Ballard',
 u'Beacon Hill',
 u'Belltown',
 u'Briarcrest',
 u'Broadview Bitter Lake',
 u'Broadway (Capitol Hill)',
 u'Cedar Park / Meadowbrook',
 u'Columbia City',
 u'Downtown (Central Business District)',
 u'East Queen Anne',
 u'Eastlake',
 u'Fairmount Park',
 u'Fauntleroy',
 u'Fremont',
 u'Gatewood',
 u'Genesee',
 u'Georgetown',
 u'Green Lake',
 u'Haller Lake',
 u'Harbor Island',
 u'Harrison / Denny Blaine',
 u'High Point',
 u'Highland Park',
 u'Highland Terrace',
 u'Industrial District',
 u'Interbay',
 u'International District',
 u'Laurelhurst / Sand Point',
 u'Leschi',
 u'Licton Springs',
 u'Lower Queen Anne',
 u'Madison Park',
 u'Madrona',
 u'Magnolia',
 u'Mann',
 u'Minor',
 u'Montlake',
 u'Mount Baker',
 u'North Admiral',
 u'North Beach Blue Ridge',
 u'North Beacon Hill',
 u'North Delridge',
 u'North Queen Anne',
 u'Northgate',
 u'Olympic Hills / Victory Heights',
 u'Phinney Ridge / Greenwood',
 u'Pike Market',
 u'Pioneer Square',
 u'Po

In [ ]:
## Add Stemmed TFIDF Vecorizer

### SnowballStemmer

In [61]:
#Snowvall Stemmer
from nltk.stem.snowball import SnowballStemmer
class StemmedTfidfVectorizer(TfidfVectorizer):
    def build_analyzer(self):
        stemmer_porter = SnowballStemmer('english')
        analyzer = super(TfidfVectorizer, self).build_analyzer()
        return lambda doc: [stemmer_porter.stem(word) for word in analyzer(doc)]    

In [62]:
stemvectorizer=StemmedTfidfVectorizer()
stemmedvectors=stemvectorizer.fit_transform(textreview)
stemmed_review_vectors=stemmedvectors.toarray()

In [63]:
from sklearn.metrics.pairwise import cosine_similarity
cos_sim = cosine_similarity(stemmed_review_vectors[0:1], stemmed_review_vectors) 

In [64]:
n=3
neighborhoods_list = df['name']
sb_stemmed_similatiry_dict={}
for i in range(len(neighborhoods_list)):
    cos_sim = cosine_similarity(stemmed_review_vectors[i:(i+1)], stemmed_review_vectors)
    order = list(cos_sim.argsort()[0][::-1][1:n])
    top_three = neighborhoods_list[order]
    sb_stemmed_similatiry_dict[neighborhoods_list[i]]=top_three.values.tolist()

### PorterStemmer

In [55]:
from nltk.stem.porter import PorterStemmer
class StemmedTfidfVectorizer(TfidfVectorizer):
    def build_analyzer(self):
        stemmer_porter = PorterStemmer()
        analyzer = super(TfidfVectorizer, self).build_analyzer()
        return lambda doc: [stemmer_porter.stem(word) for word in analyzer(doc)]    

In [56]:
stemvectorizer=StemmedTfidfVectorizer()
stemmedvectors=stemvectorizer.fit_transform(textreview)
stemmed_review_vectors=stemmedvectors.toarray()

In [58]:
from sklearn.metrics.pairwise import cosine_similarity
cos_sim = cosine_similarity(stemmed_review_vectors[0:1], stemmed_review_vectors) 

In [59]:
n=3
neighborhoods_list = df['name']
ps_stemmed_similatiry_dict={}
for i in range(len(neighborhoods_list)):
    cos_sim = cosine_similarity(stemmed_review_vectors[i:(i+1)], stemmed_review_vectors)
    order = list(cos_sim.argsort()[0][::-1][1:n])
    top_three = neighborhoods_list[order]
    ps_stemmed_similatiry_dict[neighborhoods_list[i]]=top_three.values.tolist()

In [70]:
{k: sb_stemmed_similatiry_dict[k] for k in sb_stemmed_similatiry_dict.keys()[:2]}

{u'Cedar Park / Meadowbrook': [u'Green Lake',
  u'Downtown (Central Business District)'],
 u'Stevens (Capitol Hill)': [u'Lower Queen Anne',
  u'Downtown (Central Business District)']}

In [71]:
{k: ps_stemmed_similatiry_dict[k] for k in ps_stemmed_similatiry_dict.keys()[:2]}

{u'Cedar Park / Meadowbrook': [u'Green Lake',
  u'Downtown (Central Business District)'],
 u'Stevens (Capitol Hill)': [u'Lower Queen Anne',
  u'Downtown (Central Business District)']}

In [31]:
n=3
neighborhoods_list = df['name']
similatiry_dict={}
for i in range(len(neighborhoods_list)):
    cos_sim = cosine_similarity(review_vectors[i:(i+1)], review_vectors)
    order = list(cos_sim.argsort()[0][::-1][1:n])
    top_three = neighborhoods_list[order]
    similatiry_dict[neighborhoods_list[i]]=top_three.values.tolist()

In [36]:
similatiry_dict['University District']

[u'Ravenna / Bryant', u'Montlake']

In [44]:
similatiry_dict['Alki']

[u'North Admiral', u'North Beach Blue Ridge']

In [48]:
stemmed_similatiry_dict['Alki']

[u'North Admiral', u'Downtown (Central Business District)', u'Green Lake']

In [49]:
stemmed_similatiry_dict['Downtown (Central Business District)']

[u'Pike Market', u'Broadway (Capitol Hill)', u'Green Lake']